<a href="https://colab.research.google.com/github/ShivathejS/project/blob/main/fakenewspred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [ ]:
df0= pd.read_csv('fake.csv')
df1= pd.read_csv('true.csv')


In [ ]:
df0["class"]=0
df1["class"]=1

In [ ]:
data_fakemantesting = df0.tail(10)
for i in range(23480, 23470,-1):
  df0.drop([i], axis = 0, inplace = True)

data_realmantesting = df1.tail(10)
for i in range(21416, 21406,-1):
  df1.drop([i], axis = 0, inplace = True)



In [ ]:
data_fakemantesting['class']=0
data_realmantesting['class']=1

/tmp/ipython-input-614045806.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fakemantesting['class']=0
/tmp/ipython-input-614045806.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_realmantesting['class']=1


In [ ]:
data_merged = pd.concat([df0,df1], axis=0)

In [ ]:
data_merged = data_merged.drop(['title','subject','date'], axis = 1)


In [ ]:
data_merged = data_merged.sample(frac = 1)
data_merged.head()

,text,class
16584,This report is so outrageous that we could har...,0
3789,WASHINGTON (Reuters) - President Donald Trump ...,1
20501,What happens when a black police officer sings...,0
11438,AROUND 70 per cent of female refugees in north...,0
19195,LONDON (Reuters) - A financial transaction tax...,1


In [ ]:
data_merged.reset_index(inplace = True)
data_merged.drop(['index'], axis = 1, inplace = True)

In [ ]:
def wordrem(text):
  text = text.lower()
  text =  re.sub('\[.*?\]','',text)
  text =  re.sub("\\W", " ", text)
  text = re.sub('https?://\S+|www\.\S+', '', text)
  text = re.sub('<.*?>+', '', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  text =  re.sub('\n', '', text)
  text = re.sub('\w*\d\w*', '', text)
  return text

<>:3: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:9: SyntaxWarning: invalid escape sequence '\w'
<>:3: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:9: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipython-input-2236535936.py:3: SyntaxWarning: invalid escape sequence '\['
  text =  re.sub('\[.*?\]','',text)
/tmp/ipython-input-2236535936.py:5: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub('https?://\S+|www\.\S+', '', text)
/tmp/ipython-input-2236535936.py:9: SyntaxWarning: invalid escape sequence '\w'
  text = re.sub('\w*\d\w*', '', text)


In [ ]:
data_merged['text'] = data_merged['text'].apply(wordrem)

In [ ]:
x = data_merged['text']
y = data_merged['class']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.25)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vector = TfidfVectorizer()
xv_train = vector.fit_transform(x_train)
xv_test =  vector.transform(x_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(xv_train, y_train)

LogisticRegression()

In [ ]:
pred_lr = lr.predict(xv_test)

In [ ]:
lr.score(xv_test, y_test)

0.9863636363636363

In [ ]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5787
           1       0.98      0.99      0.99      5433

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [ ]:
from sklearn.tree import DecisionTreeClassifier
dc = DecisionTreeClassifier()
dc.fit(xv_train, y_train)

DecisionTreeClassifier()

In [ ]:
pred_dc = dc.predict(xv_test)

In [ ]:
dc.score(xv_test, y_test)

0.9944741532976827

In [ ]:
def output_lable(n):
  if n == 0:
    return "Fake News"
  elif n == 1:
    return "True dis"

def manual_testing(news):
  testing_news =  {"text":[news]}
  new_def_test = pd.DataFrame(testing_news)
  new_def_test["text"] = new_def_test["text"].apply(wordrem)
  new_x_test = new_def_test["text"]
  new_xv_test = vector.transform(new_x_test)
  pred_lr = lr.predict(new_xv_test)
  pred_dc = dc.predict(new_xv_test)

  return print("\n\nLR Prediction: {} \n DT Prediction:{}".format(output_lable(pred_lr[0]),
                                            output_lable(pred_dc[0])))

In [ ]:
news = str(input())
manual_testing(news)

WASHINGTON (Reuters) - Democrat Doug Jones, who scored an upset victory last week in the U.S. Senate race in Alabama, said on Sunday he did not believe President Donald Trump needed to resign over sexual misconduct allegations against him. â€œI donâ€™t think that the president ought to resign at this point,â€ Jones told CNNâ€™s State of the Union program. â€œThose allegations were made before the election, and so people had an opportunity to judge before that election. I think we need to move on and not get distracted by those issues.â€ More than a dozen women have accused Trump of making unwanted sexual advances against them before he entered politics.  Accusations of sexual harassment against high-profile men in politics, media and the entertainment industry have put a new spotlight on the allegations against Trump, and several Democratic senators have called on him to resign. Trump and White House officials have denied the allegations. The accusations emerged during the 2016 presi